# MiniGrid Environment

The minigrid environments provide a number of "simple" environments available as both gridworlds and pixelated images. They enable us to avoid doing CNN feature visualisation analysis while still studying problems that might involve search like notions. 

Try out the environment by running the following command:


```bash
python -m minigrid.manual_control
```

Later we can benchmark against torch-rl 

In [6]:
import gymnasium as gym
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper

env = gym.make('MiniGrid-Empty-8x8-v0')
env = RGBImgPartialObsWrapper(env) # Get pixel observations
env = ImgObsWrapper(env) # Get rid of the 'mission' field
obs, _ = env.reset() # This now produces an RGB tensor only
# obs

In [ ]:
import torch as t 
import plotly.express as px
obs = t.tensor(obs)
obs.shape
px.imshow(obs)

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [ ]:
env = gym.make('MiniGrid-Empty-8x8-v0')
env = RGBImgPartialObsWrapper(env) # Get pixel observations
env = ImgObsWrapper(env) # Get rid of the 'mission' field
obs, _ = env.reset() # This now produces an RGB tensor only

# take several actions, store the observations, actions, returns and timesteps
all_obs = []
all_actions = []
all_returns = []
all_timesteps = []


for i in range(10):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    all_obs.append(obs)
    all_actions.append(action)
    all_returns.append(reward)
    all_timesteps.append(i)

# convert to tensors.unsqueeze(0)
all_obs = t.tensor(all_obs)
all_actions = t.tensor(all_actions).reshape(-1, 1)
all_returns = t.tensor(all_returns)
all_returns = t.randn((10, 1))
all_returns_to_go = all_returns.flip(0).cumsum(0).flip(0).reshape(-1, 1)
all_timesteps = t.tensor(all_timesteps).reshape(-1, 1)

/var/folders/qd/04zj2h4n2pvc87x_7w1z97_r0000gn/T/ipykernel_66398/941578228.py:22: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)



# CNN 

In [ ]:
# for the grid world environment we will a small CNN to extract features from the image
# we will use the same CNN as in the original paper

import torch as t
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange


class StateEncoder(nn.Module):
    def __init__(self, n_embed):
        super(StateEncoder, self).__init__()
        self.n_embed = n_embed
        # input has shape 56 x 56 x 3
        # output has shape 1 x 1 x 512
        self.conv1 = nn.Conv2d(3, 32, 8, stride=4, padding=0) # 56 -> 13
        self.conv2 = nn.Conv2d(32, 64, 4, stride=2, padding=0) # 13 -> 5
        self.conv3 = nn.Conv2d(64, 64, 3, stride=1, padding=0) # 5 -> 3
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(576, n_embed)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.flatten(x)
        x = self.fc(x)
        x = F.relu(x)
        return x

# we will use the same CNN as in the original paper
cnn = StateEncoder(64).to("cpu")
x = obs.unsqueeze(0).to(t.float32)
x = rearrange(x, 'b h w c-> b c h w')
cnn(x)

tensor([[ 2.2918,  2.3031,  0.0000,  0.0000,  0.0000,  0.0000,  4.8692,  0.0000,
          0.0000,  0.0000,  1.3046,  3.2425,  0.0000,  2.4428,  0.0000,  5.8740,
          0.9391,  5.1203,  2.1983,  0.0000,  0.0000,  0.0000,  1.6969,  7.1444,
          0.0000,  1.0069,  1.9155,  0.0000,  1.5764,  2.6883,  0.0000,  5.2671,
          0.0000,  0.0000,  3.8132,  0.0000,  0.0000,  4.7259,  0.5788,  0.0000,
          0.8931,  0.0000,  0.0000,  3.6706,  0.0000,  0.7813,  3.8906,  0.0000,
          0.9702,  2.0933,  0.0000, 12.0165,  2.1961,  0.0000,  4.0366,  2.6068,
          7.6517,  0.0000,  0.7258,  5.9120,  0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=<ReluBackward0>)

For reference: https://github.com/kzl/decision-transformer/blob/master/atari/mingpt/model_atari.py

In [ ]:
import torch as t 
import gymnasium as gym
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper

env = gym.make('MiniGrid-Empty-8x8-v0')
env = RGBImgPartialObsWrapper(env) # Get pixel observations
env = ImgObsWrapper(env) # Get rid of the 'mission' field
obs, _ = env.reset() # This now produces an RGB tensor only

# take several actions, store the observations, actions, returns and timesteps
all_obs = []
all_actions = []
all_returns = []
all_timesteps = []


for i in range(10):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    all_obs.append(obs)
    all_actions.append(action)
    all_returns.append(reward)
    all_timesteps.append(i)

# convert to tensors.unsqueeze(0)
all_obs = t.tensor(all_obs).to(t.float32).unsqueeze(0)
all_actions = t.tensor(all_actions).reshape(-1, 1).unsqueeze(0)
all_returns = t.randn((10, 1))
all_returns_to_go = all_returns.flip(0).cumsum(0).flip(0).reshape(-1, 1).unsqueeze(0)
all_timesteps = t.tensor(all_timesteps).reshape(-1, 1).unsqueeze(0)


# Train a decision transformer on minigrid

I thought it might be easy to sample trajectories from a random agent on minigrid and train on these. 

The problem with this approach is that it's just massively too slow. We code that does parallelized environments and agents which sample from "solution trajectories" more often than random agents.

In [ ]:
import numpy as np 
from typing import Union
ActType = Union[int, np.ndarray]

class Agent:
    '''Base class for agents in a multi-armed bandit environment (you do not need to add any implementation here)'''

    rng: np.random.Generator

    def __init__(self, num_arms: int, seed: int):
        self.num_arms = num_arms
        self.reset(seed)

    def get_action(self) -> ActType:
        raise NotImplementedError()

    def observe(self, action: ActType, reward: float, info: dict) -> None:
        pass

    def reset(self, seed: int) -> None:
        self.rng = np.random.default_rng(seed)

class RandomAgent(Agent):
    def __init__(self, env):
        self.env = env
    def get_action(self):
        return self.env.action_space.sample()

def run_episode(env: gym.Env, agent: Agent, seed: int):
    rewards = []
    actions = []
    states = []
    env.reset(seed=seed)
    agent.reset(seed=seed)
    done = False
    truncated = False
    while ((not done) and (not truncated)):
        arm = agent.get_action()
        actions.append(arm)
        (obs, reward, done, truncated, info) = env.step(arm)
        agent.observe(arm, reward, info)
        states.append(obs)
        rewards.append(reward)
    rewards = np.array(rewards, dtype=float)
    actions = np.array(actions, dtype=int)

    return rewards, np.array(states), actions

env = gym.make('MiniGrid-Empty-5x5-v0')
env = RGBImgPartialObsWrapper(env) # Get pixel observations
env = ImgObsWrapper(env) # Get rid of the 'mission' field
# add a truncation wrapper
agent = RandomAgent(env)

reward_trajs = []
states_trajs = []
actions_trajs = []
for event in range(100):
    reward_traj, states_traj, actions_traj = run_episode(env, agent, seed=i)
    reward_trajs.append(reward_traj)
    states_trajs.append(states_traj)
    actions_trajs.append(actions_traj)

# gym.vector.SyncVectorEnv(
#     env_fns=[lambda: gym.make('MiniGrid-Empty-5x5-v0') for _ in range(10)],
# )

reward_trajs = np.array(reward_trajs)
states_trajs = np.array(states_trajs)
actions_trajs = np.array(actions_trajs)


/var/folders/qd/04zj2h4n2pvc87x_7w1z97_r0000gn/T/ipykernel_66398/3137470864.py:68: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/var/folders/qd/04zj2h4n2pvc87x_7w1z97_r0000gn/T/ipykernel_66398/3137470864.py:69: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/var/folders/qd/04zj2h4n2pvc87x_7w1z97_r0000gn/T/ipykernel_66398/3137470864.py:70: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you mus

In [ ]:
import plotly.express as px 
lengths = [len(traj) for traj in reward_trajs]
px.histogram(lengths)

In [ ]:
# import decision transformer
from src.decision_transformer import DecisionTransformer

# let's try an example with a single trajectory
reward_traj = reward_trajs[0]
states_traj = states_trajs[0]
actions_traj = actions_trajs[0]

reward_traj

rtg = np.flip(reward_traj).cumsum(0)

decision_transformer = DecisionTransformer(env, max_game_length= 10*5)

logits, _ = decision_transformer(
    states = t.tensor(states_traj).to(t.float32).unsqueeze(0),
    actions = t.tensor(actions_traj).unsqueeze(0).unsqueeze(-1),
    rtgs = t.tensor(rtg).unsqueeze(0).unsqueeze(-1),
    timesteps = t.tensor(np.arange(len(reward_traj))).unsqueeze(0).unsqueeze(-1)
)

IndexError: index out of range in self

# See if I can load a replay buffer from D4RL

The original paper appears to use loaded trajectories from d4rl. We can look at these trajectories, their format and structure for reference when storing our own trajectories

In [ ]:
import gymnasium as gym
import d4rl
import minigrid
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper
from warnings import simplefilter
simplefilter(action='ignore', category=DeprecationWarning)
env = gym.make('maze2d-eval-medium-v1')
# _ = env.reset() # This now produces an RGB tensor only
env.get_dataset()

NameNotFound: Environment maze2d-eval-medium doesn't exist. 

After a huge amount of work this seems not good. I will need to train my own agent.

# Training on PPO and Storing Trajectories


### Trying Torch-RL

```bash
python3 -m scripts.train --algo ppo --env MiniGrid-DoorKey-5x5-v0 --model DoorKey --save-interval 10 --frames 80000
python3 -m scripts.visualize --env MiniGrid-DoorKey-5x5-v0 --model DoorKey
python3 -m scripts.evaluate --env MiniGrid-DoorKey-5x5-v0 --model DoorKey
```

Unfortunately, it doesn't appear super simple to use these models because we need to actually load all their classes and stuff. Let's use Callums.

## CartPole

In [ ]:
from src.ppo.train import train_ppo
from src.ppo.utils import PPOArgs
from src.utils import TrajectoryWriter
import warnings 
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category= DeprecationWarning)

    args = PPOArgs(
        exp_name = 'CartPole-v1',
        env_id = 'CartPole-v1',
        num_envs = 10,
        track = False,
        wandb_project_name="PPO-MiniGrid-test with cartpole",
        capture_video=True,
        cuda = False,
        total_timesteps=100000,
        max_steps=None)

    trajectory_writer = TrajectoryWriter(args.trajectory_path, args)

    ppo = train_ppo(args, trajectory_writer)

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/traitlets/traitlets.py:1016: DeprecationWarning: Widget._active_widgets is deprecated.
  value = getattr(cls, name)
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/traitlets/traitlets.py:1016: DeprecationWarning: Widget._widget_types is deprecated.
  value = getattr(cls, name)
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/traitlets/traitlets.py:1016: DeprecationWarning: Widget.widget_types is deprecated.
  value = getattr(cls, name)
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/traitlets/traitlets.py:1016: DeprecationWarning: Widget.widgets is deprecated.
  value = getattr(cls, name)
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/traitlets/traitlets.py:1016: DeprecationWarn

Output(layout=Layout(padding='15px'))

100%|██████████| 78/78 [00:08<00:00,  8.83it/s]

Trajectory written to trajectories/CartPole-v1.pkl


## Minigrid Env

In [3]:
from src.ppo.train import train_ppo
from src.ppo.utils import PPOArgs
from src.utils import TrajectoryWriter
from src.environments import make_env
import warnings 
import wandb

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category= DeprecationWarning)

    args = PPOArgs(
        exp_name = 'MiniGrid-Dynamic-Obstacles-8x8-v0',
        env_id = 'MiniGrid-Dynamic-Obstacles-8x8-v0',
        num_envs = 8,
        num_steps=128,
        track = True,
        wandb_project_name="PPO-MiniGrid",
        capture_video=True,
        cuda = False,
        total_timesteps=600000,
        max_steps=200,
        fully_observed=False)

    trajectory_writer = TrajectoryWriter(args.trajectory_path, args)

    ppo = train_ppo(args, trajectory_writer=trajectory_writer, make_env=make_env)

wandb.finish()

A Jupyter Widget

Thread SenderThread:
Traceback (most recent call last):
  File "/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/internal/internal.py", line 309, in _process
    self._sm.send(record)
  File "/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/internal/sender.py", line 305, in send
    send_handler(record)
  File "/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/internal/sender.py", line 319, in send_request
    send_handler(record)
  File "/U

Problem at: /Users/josephbloom/GithubRepositories/DecisionTransformerInterpretability/src/ppo/train.py 58 train_ppo


Traceback (most recent call last):
  File "/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 1078, in init
    run = wi.init()
  File "/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 697, in init
    result = handle.wait(
  File "/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/wandb/sdk/lib/mailbox.py", line 259, in wait
    raise MailboxError("transport failed")
wandb.errors.MailboxError: transport failed
wandb: ERROR Abnormal program exit


Exception: problem

# Turning the stored trajectories into a dataset

In [1]:
from src.utils import TrajectoryReader
from src.offline_dataset import TrajectoryLoader
import numpy as np
import torch as t 
from einops import rearrange
import random


path = "/Users/josephbloom/GithubRepositories/DecisionTransformerInterpretability/trajectories/MiniGrid-DoorKey-8x8-v0.pkl"
trajectory_data_set = TrajectoryLoader(path, pct_traj=0.1, device="cpu")

{'args': {'exp_name': 'MiniGrid-DoorKey-8x8-v0', 'seed': 1, 'cuda': False, 'track': True, 'wandb_project_name': 'PPO-MiniGrid', 'wandb_entity': None, 'capture_video': True, 'env_id': 'MiniGrid-DoorKey-8x8-v0', 'total_timesteps': 1000000, 'learning_rate': 0.00025, 'num_envs': 4, 'num_steps': 128, 'gamma': 0.99, 'gae_lambda': 0.95, 'num_minibatches': 4, 'update_epochs': 4, 'clip_coef': 0.2, 'ent_coef': 0.01, 'vf_coef': 0.5, 'max_grad_norm': 0.5, 'max_steps': 1000, 'trajectory_path': 'trajectories/MiniGrid-DoorKey-8x8-v0.pkl', 'fully_observed': False}, 'time': 1672685622.659515}


In [2]:
s, a, r, d, rtg, timesteps, mask = trajectory_data_set.get_batch(10)

## Visualizing a Trajectory

In [8]:
import gymnasium as gym
import plotly.express as px
from src.visualization import render_minigrid_observations, render_minigrid_observation

from minigrid.core.constants import IDX_TO_OBJECT
import numpy as np
import torch

def find_agent(observation):
    height = observation.shape[0]
    width = observation.shape[1]
    for i in range(width):
        for j in range(height):
            object = IDX_TO_OBJECT[int(observation[j,i][0])]
            if object == 'agent':
                return j, i
    #raise Exception("Agent not found")
    return 0,0

def render_minigrid_observation(env, observation):
    if isinstance(observation, np.ndarray):
        observation = observation.copy() # so we don't edit the original object
    elif isinstance(observation, torch.Tensor):
        observation = observation.numpy().copy()

    agent_pos = find_agent(observation)
    agent_dir = observation[agent_pos[0], agent_pos[1]][2]

    observation[agent_pos[0], agent_pos[1]] = [0,0,0]

    grid, _ = env.grid.decode(observation.astype(np.uint8))
    
    i = agent_pos[0]
    j = agent_pos[1]
    
    return grid.render(32, (i,j), agent_dir=agent_dir)

def render_minigrid_observations(env, observations):
    return np.array([render_minigrid_observation(env, observation) for observation in observations])


metadata = trajectory_data_set.metadata
env = gym.make(metadata['args']['env_id'], render_mode = 'rgb_array')
print(metadata['args']['env_id'])
_, _ = env.reset()

from minigrid.core.actions import Actions

event = 4
# print([Actions(int(i)) for i in a[event][mask[event].to(t.bool)]])
# print(r[event][mask[event].to(t.bool)])
# print(rtg[event][t.tensor([0])+mask[event].to(t.bool)])
imgs = render_minigrid_observations(env, s[event][mask[event].to(t.bool)])
fig = px.imshow(imgs, animation_frame=0)
fig.show()

MiniGrid-DoorKey-8x8-v0


/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



We can see that the actions/states/rewards are now indexed so that action[0] is taken after state[0] and generates reward[0]. ie: SAR, SAR, SAR

# A Training Loop

To get a training loop working, we need the following:
- the ability to do a forward pass with our decision tranformer on the data. 
- a written training loop

In terms of training details we'll use the original methods from the decision transformer paper. This means:
- train on all trajectories
- batch size of 64, 
- 128 embed dim 
- 3 layers
- 1 head
- relu activation
- 0.1 dropout
- 0.0001 learning rate
- weight decay
- max length 0f 1000?

I'll probably never both adding layer norm in though they likely have it.

Once we have a working model, we will make the following adjustments:
- less layers 
- more heads
- no dropout

Another good thing to do might be to be able to init our data loader off of the downloadable trajectories. It shouldn't be too hard to do this. 


In [15]:
# pull it all together

from src.environments import make_env

from src.decision_transformer.decision_transformer import DecisionTransformer
from src.decision_transformer.offline_dataset import TrajectoryLoader
from src.decision_transformer.train import train, test, evaluate_dt_agent

path = "trajectories/MiniGrid-DoorKey-8x8-v0.pkl"
# path = "trajectories/MiniGrid-Dynamic-Obstacles-6x6-v0.pkl"
trajectory_data_set = TrajectoryLoader(path, pct_traj=1.0, device="cpu")

# make an environment 
env_id = trajectory_data_set.metadata['args']['env_id']
env = make_env(env_id, seed = 0, idx = 0, capture_video=False, run_name = "dev2", fully_observed=False)
env = env()

# make a decision transformer
dt = DecisionTransformer(
    env = env, 
    d_model = 128,
    n_heads = 2,
    d_mlp = 256,
    n_layers = 2,
    state_embedding_type="grid", # hard-coded for now to minigrid.
    max_timestep=trajectory_data_set.metadata.get("args").get("max_steps") # Our DT must have a context window large enough
)

dt = train(dt, trajectory_data_set, env, device = "cpu", max_len = 60, batches = 200, lr = 0.001, batch_size = 128)

loss, accuracy = test(dt, trajectory_data_set, make_env, device = "cpu", max_len = 60, batch_size = 32)

print(f"loss: {loss}, accuracy: {accuracy}")

prop_completed, all_frames = evaluate_dt_agent(trajectory_data_set, dt, make_env, device = "cpu", max_len = 60, trajectories = 1000)

{'args': {'exp_name': 'MiniGrid-DoorKey-8x8-v0', 'seed': 1, 'cuda': False, 'track': True, 'wandb_project_name': 'PPO-MiniGrid', 'wandb_entity': None, 'capture_video': True, 'env_id': 'MiniGrid-DoorKey-8x8-v0', 'total_timesteps': 1000000, 'learning_rate': 0.00025, 'num_envs': 4, 'num_steps': 128, 'gamma': 0.99, 'gae_lambda': 0.95, 'num_minibatches': 4, 'update_epochs': 4, 'clip_coef': 0.2, 'ent_coef': 0.01, 'vf_coef': 0.5, 'max_grad_norm': 0.5, 'max_steps': 1000, 'trajectory_path': 'trajectories/MiniGrid-DoorKey-8x8-v0.pkl', 'fully_observed': False}, 'time': 1672849869.645534}


Training DT: 0.0012:  22%|██▎       | 45/200 [00:44<02:33,  1.01it/s]


KeyboardInterrupt: 

In [31]:
import torch as t
import plotly.express as px 
px.histogram(t.tensor(trajectory_data_set.returns))

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [23]:
import os 

videos = [i for i in os.listdir("videos/dt_eval_videos/") if i.endswith(".mp4")]
videos


['rl-video-episode-50.mp4', 'rl-video-episode-0.mp4']

# Look at Downloaded Trajectories

In [5]:
import gym 
import minigrid
import d4rl 

env = gym.make('maze2d-umaze-v1')
env.reset()
env.step(env.action_space.sample())
dataset = env.get_dataset()

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/gym/utils/seeding.py:38: DeprecationWarning: WARN: Function `rng.randn(*size)` is marked as deprecated and will be removed in the future. Please use `rng.standard_normal(size)` instead.
  deprecation(


load datafile: 100%|██████████| 8/8 [00:00<00:00, 17.95it/s]


In [7]:
dataset.keys()

dict_keys(['actions', 'infos/goal', 'infos/qpos', 'infos/qvel', 'observations', 'rewards', 'terminals', 'timeouts'])

In [9]:
dataset['observations']

array([[ 1.0856489 ,  1.9745734 ,  0.00981035,  0.02174424],
       [ 1.0843927 ,  1.97413   , -0.12562364, -0.04433781],
       [ 1.0807577 ,  1.9752754 , -0.3634883 ,  0.11453988],
       ...,
       [ 1.1328583 ,  2.8062387 , -4.484303  ,  0.09555068],
       [ 1.0883482 ,  2.8068895 , -4.4510083 ,  0.06509537],
       [ 1.0463258 ,  2.8074222 , -4.202244  ,  0.05324839]],
      dtype=float32)

In [17]:
dataset['observations'][:100].shape

(100, 4)

In [11]:
dataset['timeouts'][:100]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False])